In [1]:
import time
import pandas as pd
import numpy as np
from multiprocessing.dummy import Pool
import time
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import random
from bs4 import BeautifulSoup
import urllib.request, json, time

## Works to get stocks from traderfox

In [2]:
def get_stock_data(url, index_exchange):
    """
    Function to fetch stock information from a specified URL of the traderfox.de website
    with an specified index / exchange

    Inputs:
        url (str): one of the available urls from traderfox.de
        index_exchange (str): name of the index / exchnage from which
            the stocks should be selected
    """
    browser =  webdriver.Firefox()
    time.sleep(2)
    browser.get(url)
    time.sleep(5)
    soup = BeautifulSoup(browser.page_source,"html")
    time.sleep(5)
    names = []
    Row = soup.find('table', attrs = {'id' : 'insert-stocks'})
    for name in Row.find_all('td', attrs = {'class' : 'name'}):
            try:
                names.append(name.text)
            except:
                names.append("")

    wkns = []
    Row = soup.find('table', attrs = {'id' : 'insert-stocks'})
    for wkn in Row.find_all('td', attrs = {'data-id' : 'wkn'}):
            try:
                wkns.append(wkn.text)
            except:
                wkns.append("")

    FRAME = pd.DataFrame.from_dict(
            {'name': names,
            'wkn': wkns,
            'index' : index_exchange
            })
    time.sleep(1)
    browser.quit()
    return(FRAME)

In [ ]:

url_ls = ["https://traderfox.de/aktien/alle-amex-aktien-bestandteile"]#"https://traderfox.de/aktien/alle-nasdaq-aktien-bestandteile"]#]
#  "https://traderfox.de/aktien/alle-nyse-aktien-bestandteile"]#,
   # "https://traderfox.de/aktien/deutschland-160-bestandteile",
   # "https://traderfox.de/aktien/alle-nyse-aktien-bestandteile",
  #  "https://traderfox.de/aktien/alle-nasdaq-aktien-bestandteile",
#]
#https://traderfox.de/aktien/stoxx-europe-600-bestandteile
country_names_ls = ["AMEX"]

In [ ]:
ticker_frame = pd.DataFrame({})
for i, url in enumerate(url_ls):
    df = get_stock_data(url, country_names_ls[i])
    ticker_frame = ticker_frame.append(df)
    time.sleep(1)



## Chinese Stocks

#### instead of getting the stocks from traderfox, I collected them in a CSV. Sometimes you an find all stocks of a country / exchange to download, sometimes it is worth copy pasting them but sometimes you have to scrape them like above

In [3]:
# load and isnpect data
# I am usually lazy and just use the whole path... THIS WILL NOT WORK FOR YOU! You have to adapt your own path to the file

ticker_frame = pd.read_excel("C:/Users/Jonathan/Documents/Stock_project/stockpicking_screener/screener/data_management/scraper_chinese/Mappe 1.xlsx")
ticker_frame.head()

,Company Name,Code
0,"SHANGHAI PUDONG DEVELOPMENT BANK CO., LTD.",600000
1,"Guangzhou Baiyun International Airport Co.,ltd.",600004
2,"Dongfeng Automobile Co.,LTD",600006
3,China World Trade Center Company Ltd.,600007
4,Beijing Capital Eco-Environment Protection Gro...,600008


The following code is the adapted code from the notebook you will find in `/screener/data_management/scraper`.

I adapted it such that I will get information about the industry and it will also check the name again (Since some stocks had faulty / very complicated names for shanghai). These two infos are required for every stock (if possible)

In [4]:
def get_ticker(initial_frame):
    #url_ticker = "https://www.finanznachrichten.de/"
    url_yf_ticker = "https://finance.yahoo.com/quote/FB?p=FB"
    browser =  webdriver.Firefox()

    # MY CUSTOM EXTENSIONS THAT I USE (not needed)
    #extension_path = r"C:\Users\Jonathan\AppData\Roaming\Mozilla\Firefox\Profiles\wl4weym2.default-1633941918487\extensions\jid1-MnnxcxisBPnSXQ@jetpack.xpi"
    #browser.install_addon(extension_path, temporary=True)
    time.sleep(2)
    browser.get("about:support")
    time.sleep(5)

    name_yf = []
    industry = []
   
    try:
        browser.get(url_yf_ticker)
        time.sleep(5)
    except:
        browser.get(url_yf_ticker)
        time.sleep(5)
    try:
        browser.find_element_by_css_selector("#scroll-down-btn").click()
        time.sleep(3)
    except:
        pass
    try:
        browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
        time.sleep(4)
    except:
        pass
    for ticker in initial_frame.Code:
        time.sleep(1)
        #check again for popup and scroll
        try:
            browser.find_element_by_css_selector("#scroll-down-btn").click()
            time.sleep(1)
        except:
            pass
        try:
            browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
            time.sleep(2)
        except:
            pass

        clear = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry"))).clear()
        #time.sleep(1)
        try:
            search = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry")))
            time.sleep(1)
            search.send_keys(str(ticker))
            try:
                time.sleep(3)
                name_yf.append(str(WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-quotes-0 > div:nth-child(1) > div:nth-child(2)"))).text))
            except:
                name_yf.append("")
            time.sleep(1)
            browser.find_element_by_css_selector("#header-desktop-search-button").click()
            time.sleep(2)
            browser. execute_script("window.scrollTo(0,2500)")
            time.sleep(1)
            industry.append(str(WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div/div/div[12]/div/div/div/div/div/div/p[2]/span[2]"))).text))
        except:
            try:
                time.sleep(1)
                browser. execute_script("window.scrollTo(0,3000)")
                time.sleep(2)
                industry.append(str(WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div/div[1]/div/div[3]/div[2]/div/div/div/div/div/div[12]/div/div/div/div/div/div/p[2]/span[2]"))).text))
            except:
                industry.append("")
                

    frame = pd.DataFrame.from_dict(
        {"name_ss" : initial_frame["Company Name"],
        "index" : "Shanghai",
        "name" : name_yf,
        'ticker': initial_frame["Code"],
        'industry': industry,

        })
    time.sleep(1)
    browser.quit()
    return frame


In [5]:
# dont be scared! Firefox will open itself! :) 

# uncomment for real run!
#data_ticker_df = get_ticker(ticker_frame)

# if you want to testrun
data_ticker_df = get_ticker(ticker_frame[:5])
data_ticker_df


,name_ss,index,name,ticker,industry
0,"SHANGHAI PUDONG DEVELOPMENT BANK CO., LTD.",Shanghai,"Shanghai Pudong Development Bank Co., Ltd.",600000,Financial Services
1,"Guangzhou Baiyun International Airport Co.,ltd.",Shanghai,"Guangzhou Baiyun International Airport Co., Ltd.",600004,Industrials
2,"Dongfeng Automobile Co.,LTD",Shanghai,DongFeng Automobile Co. LTD,600006,Consumer Cyclical
3,China World Trade Center Company Ltd.,Shanghai,"China World Trade Center Co., Ltd.",600007,Real Estate
4,Beijing Capital Eco-Environment Protection Gro...,Shanghai,Beijing Capital Eco-environment Protection Gro...,600008,Industrials


##### If neccessary you can now work on the formatting of the dataframe, for Instance:

In [6]:
data_amex = data_ticker_df[["name", "ticker", "index", "industry"]]
data_amex["ticker"] = data_amex["ticker"].astype(str) + ".SS"
data_amex

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_15264/2170253028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_amex["ticker"] = data_amex["ticker"].astype(str) + ".SS"


,name,ticker,index,industry
0,"Shanghai Pudong Development Bank Co., Ltd.",600000.SS,Shanghai,Financial Services
1,"Guangzhou Baiyun International Airport Co., Ltd.",600004.SS,Shanghai,Industrials
2,DongFeng Automobile Co. LTD,600006.SS,Shanghai,Consumer Cyclical
3,"China World Trade Center Co., Ltd.",600007.SS,Shanghai,Real Estate
4,Beijing Capital Eco-environment Protection Gro...,600008.SS,Shanghai,Industrials


##### Once finished, save it!

In [ ]:
data_amex.to_csv("China_Stocks.csv")


# Additional methods from the original file under `screener/data_management/scraper` (not needed)

In [ ]:
def validate_ticker(path):

    """
    Function to update the stock information for the different exchanges / indicies
    This function is usefull to:
        - check if info is up to date
        - after adding new stocks to the {exchange / index}Stocks.csv files
        - to check that scraping was not corruped by bad internet connection or other issues
        - to validate the scraping
    
    Inputs:
        path (str): path of the file for which to perform the validation

    """
    url_ticker = "https://www.finanznachrichten.de/"
    url_yf_ticker = "https://finance.yahoo.com/quote/FB?p=FB"
    # browser =  webdriver.Firefox(executable_path='/home/jonathan/Schreibtisch/geckodriver')
    # extension_path = r"/home/jonathan/.mozilla/firefox/5xapxbqn.default-release/extensions/jid1-MnnxcxisBPnSXQ@jetpack.xpi"
    # browser.install_addon(extension_path, temporary=True)


    browser =  webdriver.Firefox()
    extension_path = r"C:\Users\Jonathan\AppData\Roaming\Mozilla\Firefox\Profiles\wl4weym2.default-1633941918487\extensions\jid1-MnnxcxisBPnSXQ@jetpack.xpi"
    browser.install_addon(extension_path, temporary=True)
    time.sleep(2)
    #extension_path = r"C:\Users\Jonathan\AppData\Roaming\Mozilla\Firefox\Profiles\wl4weym2.default-1633941918487\extensions\firefox-webext@zenmate.com.xpi"
    #browser.install_addon(extension_path, temporary=True)
    browser.maximize_window()
    browser.get("about:support")
    time.sleep(5)
    browser.get(url_ticker)
    time.sleep(5)
    file = pd.read_csv(path, encoding='utf-8')
    try:
        file.drop('Unnamed: 0', axis=1, inplace=True)
    except:
        pass
    try:
        file.drop('Unnamed: 0.1', axis = 1, index = True)
    except:
        pass

    for i, row in file.iterrows():
        if (row.ISIN) == "not_found" and (row.wkn != "-"):
            print(file.loc[i, "name"])
            try:
                time.sleep(1)
                clear = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#fnk-suche-eingabe"))).clear()
                time.sleep(random.randint(3,4))
                search = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#fnk-suche-eingabe")))
                time.sleep(random.randint(2,3))
                search.send_keys(str(row.wkn))
                time.sleep(3)
                browser.find_element_by_css_selector('#suchhilfeListe > tbody:nth-child(2) > tr:nth-child(3) > td:nth-child(2)').click()
                time.sleep(random.randint(2,3))
                file.at[i, "de_ticker"] = (WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#produkt-ticker"))).text.replace("Ticker-Symbol: ", "").lstrip())
                #time.sleep(2)
            except:
                pass
            try:
                if file.at[i, "industry"] == "not_found":
                    file.at[i, "industry"] = (WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".a > a:nth-child(2)"))).text)
                    time.sleep(3)
            except:
                pass
            try:
                if file.at[i, "ISIN"] == "not_found":
                    file.at[i, "ISIN"] = (WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#produkt-isin"))).text.replace("ISIN: ", "").lstrip())
                    time.sleep(random.randint(2,3))
            except:
                pass

    time.sleep(3)
    browser.get(url_yf_ticker)
    time.sleep(15)
    try:
        browser.find_element_by_css_selector("#scroll-down-btn").click()
        time.sleep(3)
    except:
        pass
    try:
        browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
        time.sleep(4)
    except:
        pass
    for i, row in file.iterrows():
        if row.ISIN == "not_found":
            continue
        if file.at[i, "ticker"] == "not_found":
            time.sleep(random.randint(1,2))
            try:
                browser.find_element_by_css_selector("#scroll-down-btn").click()
                time.sleep(3)
            except:
                pass
            try:
                browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
                time.sleep(4)
            except:
                pass

            clear = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry"))).clear()
            time.sleep(random.randint(3,5))
            try:
                search = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry")))
                time.sleep(1)
                search.send_keys(str(row.ISIN))
                time.sleep(5)
                file.at[i, "ticker"] = (WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-quotes-0 > div.modules_quoteLeftCol__gkCSv.modules_Ell__77DLP.modules_IbBox__2pmLe > div.modules_quoteSymbol__hpPcM.modules_Ell__77DLP.modules_IbBox__2pmLe"))).text)
            except:
                pass
    #exchange = file.index[0]
    file.to_csv("val_" + path) 

    browser.quit()


In [ ]:
validate_ticker("val_AMEX_Stocks.csv")